# Dynamic FBA

In [ ]:
%run 2.Ecoli_model.ipynb

In [1]:
from os.path import dirname, join, pardir

from cobra.io import read_sbml_model

from dfba import DfbaModel, ExchangeFlux, KineticVariable

In [2]:
%%capture --no-display
fba_model = read_sbml_model("iML1515.xml.gz")
fba_model.solver = "glpk"
dfba_model = DfbaModel(fba_model)

In [3]:
X = KineticVariable("Biomass")
Gluc = KineticVariable("Glucose")
Oxy = KineticVariable("Oxygen")
Put = KineticVariable("Putrescine")

dfba_model.add_kinetic_variables([X, Gluc, Oxy, Put])

In [12]:
mu = ExchangeFlux("BIOMASS_Ec_iML1515_core_75p37M")
v_G = ExchangeFlux("EX_glc__D_e")
v_O = ExchangeFlux("EX_o2_e")
v_P = ExchangeFlux("Ex_ptrc_e")
                   
                   
dfba_model.add_exchange_fluxes([mu, v_G, v_O, v_P])

Ignoring exchange flux BIOMASS_Ec_iML1515_core_75p37M since italready exists in the model.
Ignoring exchange flux EX_glc__D_e since italready exists in the model.
Ignoring exchange flux EX_o2_e since italready exists in the model.
Ignoring exchange flux DM_ptrc_c since it doesnot correspond to a reaction in the model.


In [9]:
dfba_model.add_rhs_expression("Biomass", mu * X)
dfba_model.add_rhs_expression("Glucose", v_G * 180.1559/1000 * X) # v_G [mmol gDW^-1 h^-1] * 0.18 g/mmol * gDW/L
dfba_model.add_rhs_expression("Oxygen", 0) # O2 is kept constant
dfba_model.add_rhs_expression("Putrescine", v_P * 88.15/1000 * X) #Putrescine -----

In [10]:
vmax_o2 = 15 # [mmol gDW^-1 h^-1]
Ko = 0.024 # mmol/L O2 Michaelis-Mentent constant
dfba_model.add_exchange_flux_lb("EX_o2_e", vmax_o2 * (Oxy / (Ko + Oxy)), Oxy)

In [ ]:
dfba_model.add_initial_conditions(
    {
        "Biomass": 0.01, # (gDW/L)
        "Glucose": 15.5, # (g/L)
        "Oxygen": 0.24,  # (mmol/L)
        "Putrescine": 0.0,  # (g/L)
    }
)
concentrations, trajectories = dfba_model.simulate(0.0, 25.0, 0.1, ["EX_glc__D_e", "EX_o2_e", "EX_ptrc_e"])